In [1]:
import os
from dotenv import load_dotenv
from scraper import fetch_website_contents
from IPython.display import Markdown, display
from openai import OpenAI
import requests

In [2]:
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

# Check the API key
if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it is in the wrong format; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


In [3]:
message = "Hello, GPT! Tell me a fun fact"

messages = [{"role": "user", "content": message}]

messages

[{'role': 'user', 'content': 'Hello, GPT! Tell me a fun fact'}]

In [4]:
openai = OpenAI()

response = openai.chat.completions.create(
    model="gpt-5-nano",
    messages=messages
)

In [6]:
response

ChatCompletion(id='chatcmpl-CqnEdTOHj0taP8VPaeXnfsk6mx4tg', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Fun fact: Octopuses have three hearts. Two pump blood to the gills, and the third pumps it to the rest of the body. When an octopus swims, the heart that delivers blood to the body stops beating, which is why they prefer crawling—swimming uses a lot of energy for them. Want another fun fact?', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1766698351, model='gpt-5-nano-2025-08-07', object='chat.completion', service_tier='default', system_fingerprint=None, usage=CompletionUsage(completion_tokens=654, prompt_tokens=15, total_tokens=669, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=576, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

In [5]:
print(response.choices[0].message.content)

Fun fact: Octopuses have three hearts. Two pump blood to the gills, and the third pumps it to the rest of the body. When an octopus swims, the heart that delivers blood to the body stops beating, which is why they prefer crawling—swimming uses a lot of energy for them. Want another fun fact?


In [11]:
ed = fetch_website_contents("https://edwarddonner.com")
print(ed)

Home - Edward Donner

Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.
We work with groundbreaking, proprietary LLMs verticalized for talent, we’ve
patented
our matching model, and our award-winning platform has happy customers and tons of press coverage.
Conne

In [28]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = """
You are a helpful assistant that analyzes the contents of a website,
and provides a short, helpful, humorous summary, ignoring text that might be navigation related.
Respond in markdown. Do not wrap the markdown in a code block - respond just with the markdown.
"""

In [29]:
user_prompt_prefix = """
Here are the contents of a website.
Provide a short summary of this website.
If it includes news or announcements, then summarize these too.

"""

In [30]:
messages = [
    {"role": "system", "content": "You are a nagging assistant"},
    {"role": "user", "content": "What is 2 + 2?"}
]

response = openai.chat.completions.create(model="gpt-5-nano", messages=messages)
response.choices[0].message.content

'4. And please don’t confuse it with “22”—that’s just concatenation, not addition. In base-10, 2 + 2 = 4. In other bases the representation can look different (e.g., in base 3, the sum would be 11).'

In [15]:
# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_prefix + website}
    ]

In [22]:
# Try this out, and then try for a few more websites

messages_for(ed)

[{'role': 'system',
  'content': '\nYou are a snarky assistant that analyzes the contents of a website,\nand provides a short, snarky, humorous summary, ignoring text that might be navigation related.\nRespond in markdown. Do not wrap the markdown in a code block - respond just with the markdown.\n'},
 {'role': 'user',
  'content': '\nHere are the contents of a website.\nProvide a short summary of this website.\nIf it includes news or announcements, then summarize these too.\n\nHome - Edward Donner\n\nHome\nConnect Four\nOutsmart\nAn arena that pits LLMs against each other in a battle of diplomacy and deviousness\nAbout\nPosts\nWell, hi there.\nI’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (\nvery\namateur) and losing myself in\nHacker News\n, nodding my head sagely to things I only half understand.\nI’m the co-founder and CTO of\nNebula.io\n.

In [31]:
# And now: call the OpenAI API. You will get very familiar with this!

def summarize(url):
    website = fetch_website_contents(url)
    response = openai.chat.completions.create(
        model = "gpt-4.1-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [32]:
summarize("https://edwarddonner.com")

"# Summary of Edward Donner's Website\n\nEdward Donner's personal site is a fascinating blend of AI innovation, coding passion, and a dash of DJ nostalgia. Ed is a tech entrepreneur and AI enthusiast who co-founded Nebula.io, a company using AI to help recruiters find and engage talent through patented models and proprietary large language models (LLMs). He also previously founded an AI startup, untapt, which was acquired in 2021.\n\nThe site features:\n- Fun projects like **Connect Four** and **Outsmart**, the latter being a clever LLM showdown arena full of intrigue and diplomacy.\n- Insightful blog posts and announcements, including upcoming AI live events, product insights on generative AI at AWS scale, and a curriculum for aspiring AI engineers.\n- A friendly invitation to connect via email or social media, plus a newsletter subscription for ongoing AI goodies.\n\nIn short: Ed is mixing AI, talent discovery, and a pinch of music geekery into a uniquely engaging space. Expect cleve

In [33]:
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [34]:
display_summary("https://edwarddonner.com")

# Summary of Edward Donner's Website

Ed Donner is a coder, AI enthusiast, DJ (though out of practice), and electronic music hobbyist who loves diving into Hacker News. He's the co-founder and CTO of Nebula.io, a company leveraging AI to help recruiters find and manage talent—a field where AI can truly shine. Previously, he started and sold an AI startup called untapt. Edward also experiments with large language models (LLMs) and has created a cool "Outsmart" arena where these LLMs battle in diplomacy and deception — sounds like AI Game of Thrones!

## News & Announcements:
- **Nov 11, 2025**: The Unique Energy of an AI Live Event
- **Sep 15, 2025**: AI in Production: Gen AI and Agentic AI on AWS at scale
- **May 28, 2025**: Be an AI Engineer and Leader: The Curriculum
- **May 18, 2025**: 2025 AI Executive Briefing

If you want to nerd out about AI or get in touch with Edward, you can reach him via email or social media, and even subscribe to his newsletter for more tech and AI adventures.

In [27]:
display_summary("https://cnn.com")

CNN’s homepage chrome: a big, loud doorway to the news empire rather than actual stories in this snippet.

- What you’re looking at: a navigation-heavy header with endless sections, live TV, sign-in options, newsletters, and topic Follow features. It’s basically the portal, not the article.
- News/announcements in this extract: none. The only “news” vibe is the label Breaking News, plus topic names (e.g., Ukraine-Russia War, Israel-Hamas War) without any specific stories to summarize.
- Snarky takeaway: if you want the actual news, you’ll have to click around the menu and hope a story finally loads. The page here is more about navigational bloat than breaking headlines.

In [35]:
# Step 1: Create your prompts

system_prompt = "act as a world class poet of Nigeria origin"
user_prompt = "write a poem about the rise of Nigeria from it's current state"

# Step 2: Make the messages list

messages = [
    {'role': 'system', 'content': system_prompt},
    {'role': 'user', 'content': user_prompt}
] # fill this in

# Step 3: Call OpenAI
response = openai.chat.completions.create(model='gpt-4o-mini', messages=messages)

In [36]:
Markdown(response.choices[0].message.content)

**Awakening: The Rise of Nigeria**

In the cradle of the Niger, dreams entwined with the sun,  
A tapestry of nations, where every thread is spun.  
From the echoes of the ancients, to the pulse of bustling streets,  
We rise from shadows softly, as the rhythm of time beats.  

O land of vibrant colors, where cultures dance and blend,  
In the canvas of your history, every voice weaves forth a mend.  
Through trials that have shaped you, like rivers winding through stone,  
Resilience flows within us, a strength we’ve always known.  

Once shrouded in the darkness of a tempest’s heavy hand,  
We emerged from the ashes, like a phoenix on the land.  
Oh, Nigeria, your heart still thunders beneath the layers of strife,  
In the womb of your potential lies the promise of new life.  

The youth, like seeds of promise, strewn in earth rich and warm,  
With dreams that rise like tides, they weather every storm.  
From the hustle of Lagos to the quiet of the north,  
Innovation sparks a flame, giving hope a fresh rebirth.  

With hands outstretched in unity, let’s weave the tapestry,  
Of every tribe and tongue, a proud mosaic to see.  
In harmony, we gather, breathing life into the tale,  
A chorus of tomorrow, where our spirits shall prevail.  

As the sun sets on the struggles, painting skies of gold anew,  
We rise, the dawn within us, igniting paths to pursue.  
Together, we’ll reimagine, a nation bold and free,  
From the ashes of the past, arises Nigeria’s decree.  

So let the drums of fervor echo through the land once more,  
In the rhythm of our journey, we’ll find what we’re here for.  
With courage forged in fire, we’ll climb each mountain high,  
In the pride of every heartbeat, let our spirit fly.  

Awake, O Nigeria, to the dreams that call your name,  
From the depths of despair, we’ll rise within the flame.  
For in every single story, both joy and pain impart,  
Lies the strength of our beginnings, the fire of our heart.  

In [37]:
# Create OpenAI client
openai = OpenAI()

response = openai.chat.completions.create(model="gpt-5-nano", messages=[{"role": "user", "content": "Tell me a fun fact"}])

response.choices[0].message.content

'Fun fact: A group of flamingos is called a “flamboyance.” Their pink color comes from carotenoid pigments in their diet, like shrimp and algae. Want another fun fact on a different topic?'

In [39]:
Markdown(response.choices[0].message.content)

Fun fact: A group of flamingos is called a “flamboyance.” Their pink color comes from carotenoid pigments in their diet, like shrimp and algae. Want another fun fact on a different topic?

### Using Ollama

In [41]:
requests.get('http://localhost:11434').content

b'Ollama is running'

In [42]:
!ollama pull llama3.2

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest 
pulling dde5aa3fc5ff:   0% ▕                  ▏ 2.2 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff:   0% ▕                  ▏ 4.7 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff:   1% ▕                  ▏  11 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff:   1% ▕                  ▏  18 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff:   1% ▕                  ▏  22 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff:   1% ▕                  ▏  29 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff:   2% ▕                  ▏  35 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff:   2% ▕                  ▏  37 MB/2.0 GB                  pulling manifest 
pulling dde5aa3f

In [43]:
llama = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

In [44]:
response = llama.chat.completions.create(model="llama3.2", messages=[{"role": "user", "content": "Tell me a fun fact"}])

response.choices[0].message.content

"Did you know that honey never expires? Archaeologists have found pots of honey in ancient Egyptian tombs that are over 3,000 years old and still perfectly edible! Honey's unique combination of acidity, water content, and presence of hydrogen peroxide make it virtually impossible for bacteria to grow, keeping it forever fresh. Isn't that sweet?"